# **LLama 3.1-8B (base version)**

Code was adapted based on an example from [Link to Source](https://github.com/adidror005/youtube-videos/blob/main/old_videos/LLAMA_3_Fine_Tuning_for_Sequence_Classification_Actual_Video.ipynb)

## **2 Labels only, no LoRa**

In [1]:
# install packages
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install pyarrow==18.1.0
!pip install evaluate
!pip install --upgrade wandb

  Using cached transformers-4.51.1-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.51.1-py3-none-any.whl (10.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
adapters 1.1.0 requires transformers~=4.47.1, but you have transformers 4.51.1 which is incompatible.


In [2]:
# import packages

import numpy as np
import pandas as pd
import os
import random
import evaluate
import functools # ??
from tqdm import tqdm
import bitsandbytes as bnb
import wandb

import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import Dataset, DatasetDict
from peft import LoraConfig, PeftConfig, prepare_model_for_kbit_training, get_peft_model

from trl import SFTTrainer
from trl import setup_chat_format

import transformers
from transformers import (AutoModelForCausalLM,
                          AutoModelForSequenceClassification,
                        AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                            Trainer,
                            DataCollatorWithPadding,
                          pipeline, 
                          logging)

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix,
                            f1_score, balanced_accuracy_score)

from peft import PeftModel


In [3]:
import torch
torch.cuda.empty_cache()
torch.cuda.is_available()
#torch.cuda.device_count()

True

## **Authenticate for Hugging Face**

In [4]:
# Hugging face access

from huggingface_hub import login
with open("../../../../login/hf_key.txt", 'r') as f: 
    HF_TOKEN = str(f.read())
    
login(token = HF_TOKEN)

In [5]:

with open("../../../../login/wandb.txt", 'r') as f: 
    WB_TOKEN = str(f.read())

wandb.login(key=WB_TOKEN)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc
wandb: Currently logged in as: elena-solar (elena-solar-university-of-konstanz) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## **Data**

In [7]:
# loading the data
import pandas as pd
data = pd.read_csv("../../../../data/debagree_new_preprocessing_com_rep.csv")
data = data[["label", "body_parent", "body_child", "msg_id_parent", "msg_id_child", "subreddit", "datetime", "exact_time"]].sort_values(by = "exact_time").reset_index(drop = True)

# keep integer labels
data['target'] = data['label']

# for readability, recode labels
int_to_label = {2: "agree", 1 : "neutral", 0 : "disagree"}
data.replace({"label": int_to_label}, inplace = True)

data

,label,body_parent,body_child,msg_id_parent,msg_id_child,subreddit,datetime,exact_time,target
0,neutral,"I live in rural Saskatchewan, Canada. We have ...",I'm in NE USA we've had 3 in two years...all e...,cnddov1,cndj2gv,climate,03/01/2015 23:18,1420327135,1
1,neutral,"I live in rural Saskatchewan, Canada. We have ...",One hundred year flood just means a one in one...,cnddov1,cndkpy7,climate,04/01/2015 00:10,1420330231,1
2,neutral,Convince her of what? That it's happening or t...,That anthropocentric climate change is actuall...,cndnlrd,cndnsxt,climate,04/01/2015 01:45,1420335952,1
3,disagree,I think this prediction is about as valid as s...,It's January. Literally no one said it would b...,cndl5x4,cndybsy,climate,04/01/2015 08:01,1420358465,0
4,disagree,"Mann hasn't been honest in decades, so I'm cur...",There have been a dozen re-constructions of Ma...,cne462t,cne89ej,climate,04/01/2015 17:45,1420393544,0
...,...,...,...,...,...,...,...,...,...
42838,neutral,Not trying to spark an argument but a legitima...,Keeping in mind that the Palestinians killed m...,gyo197v,gyotff1,Republican,19/05/2021 12:36,1621427788,1
42839,agree,Y'all saw Guilianis hail Mary right? Get his s...,"Same I want these assholes in jail, full stop....",gynfsu4,gyp3u39,democrats,19/05/2021 13:56,1621432578,2
42840,agree,Why don't I see ads holding Republicans accoun...,"Yeah, I agree with the goal of this post but n...",gyn6nzm,gyp5vzw,democrats,19/05/2021 14:11,1621433471,2
42841,agree,How about ... no? This is strange. Community o...,"I know, it feels strange too. We wouldn't hold...",gyp71o7,gyp7en6,BlackLivesMatter,19/05/2021 14:21,1621434116,2


In [8]:
# adapt true labels
labels_2 = []
target_new = []

for idx, row in data.iterrows():
    if row['label'] in ['neutral', 'agree']:
        labels_2.append('no_disagreement')
        target_new.append(0)
    else:
        labels_2.append('disagree')
        target_new.append(1)
        
data['label_2'] = labels_2
data['target'] = target_new
data

,label,body_parent,body_child,msg_id_parent,msg_id_child,subreddit,datetime,exact_time,target,label_2
0,neutral,"I live in rural Saskatchewan, Canada. We have ...",I'm in NE USA we've had 3 in two years...all e...,cnddov1,cndj2gv,climate,03/01/2015 23:18,1420327135,0,no_disagreement
1,neutral,"I live in rural Saskatchewan, Canada. We have ...",One hundred year flood just means a one in one...,cnddov1,cndkpy7,climate,04/01/2015 00:10,1420330231,0,no_disagreement
2,neutral,Convince her of what? That it's happening or t...,That anthropocentric climate change is actuall...,cndnlrd,cndnsxt,climate,04/01/2015 01:45,1420335952,0,no_disagreement
3,disagree,I think this prediction is about as valid as s...,It's January. Literally no one said it would b...,cndl5x4,cndybsy,climate,04/01/2015 08:01,1420358465,1,disagree
4,disagree,"Mann hasn't been honest in decades, so I'm cur...",There have been a dozen re-constructions of Ma...,cne462t,cne89ej,climate,04/01/2015 17:45,1420393544,1,disagree
...,...,...,...,...,...,...,...,...,...,...
42838,neutral,Not trying to spark an argument but a legitima...,Keeping in mind that the Palestinians killed m...,gyo197v,gyotff1,Republican,19/05/2021 12:36,1621427788,0,no_disagreement
42839,agree,Y'all saw Guilianis hail Mary right? Get his s...,"Same I want these assholes in jail, full stop....",gynfsu4,gyp3u39,democrats,19/05/2021 13:56,1621432578,0,no_disagreement
42840,agree,Why don't I see ads holding Republicans accoun...,"Yeah, I agree with the goal of this post but n...",gyn6nzm,gyp5vzw,democrats,19/05/2021 14:11,1621433471,0,no_disagreement
42841,agree,How about ... no? This is strange. Community o...,"I know, it feels strange too. We wouldn't hold...",gyp71o7,gyp7en6,BlackLivesMatter,19/05/2021 14:21,1621434116,0,no_disagreement


In [9]:
# dictionary for conversions
label_dict = {1: "disagree", 0 : "no_disagreement"}


In [10]:
# make text

def create_training_data(data):

    result = []

    for idx, row in data.iterrows():
        #system_prompt = """You are a classification chatbot. Analyze a Reddit comment and its reply. Determine if the reply explicitly and unambiguously disagrees with the comment. Respond with 'disagree' for clear disagreement or 'no_disagreement' if the reply adds information, qualifies the statement, offers an alternative perspective, or leaves any doubt about disagreement. Your response must strictly be 'disagree' or 'no_disagreement'."""
        comment = row["body_parent"]
        reply = row["body_child"]
        label = row["label_2"]
        target = row["target"]
        result.append({'comment' : comment, 'reply': reply, 'label' : label, 'target' : target})
    
    return result

# save data
df = pd.DataFrame(create_training_data(data))
df

,comment,reply,label,target
0,"I live in rural Saskatchewan, Canada. We have ...",I'm in NE USA we've had 3 in two years...all e...,no_disagreement,0
1,"I live in rural Saskatchewan, Canada. We have ...",One hundred year flood just means a one in one...,no_disagreement,0
2,Convince her of what? That it's happening or t...,That anthropocentric climate change is actuall...,no_disagreement,0
3,I think this prediction is about as valid as s...,It's January. Literally no one said it would b...,disagree,1
4,"Mann hasn't been honest in decades, so I'm cur...",There have been a dozen re-constructions of Ma...,disagree,1
...,...,...,...,...
42838,Not trying to spark an argument but a legitima...,Keeping in mind that the Palestinians killed m...,no_disagreement,0
42839,Y'all saw Guilianis hail Mary right? Get his s...,"Same I want these assholes in jail, full stop....",no_disagreement,0
42840,Why don't I see ads holding Republicans accoun...,"Yeah, I agree with the goal of this post but n...",no_disagreement,0
42841,How about ... no? This is strange. Community o...,"I know, it feels strange too. We wouldn't hold...",no_disagreement,0


In [11]:
df['prompt'] = None

def make_prompt(row):

    prompt = "Comment: " + row["comment"] + "; Reply: " + row["reply"]

    return prompt



df['prompt'] = df.apply(lambda row: make_prompt(row), axis = 1)
df

,comment,reply,label,target,prompt
0,"I live in rural Saskatchewan, Canada. We have ...",I'm in NE USA we've had 3 in two years...all e...,no_disagreement,0,"Comment: I live in rural Saskatchewan, Canada...."
1,"I live in rural Saskatchewan, Canada. We have ...",One hundred year flood just means a one in one...,no_disagreement,0,"Comment: I live in rural Saskatchewan, Canada...."
2,Convince her of what? That it's happening or t...,That anthropocentric climate change is actuall...,no_disagreement,0,Comment: Convince her of what? That it's happe...
3,I think this prediction is about as valid as s...,It's January. Literally no one said it would b...,disagree,1,Comment: I think this prediction is about as v...
4,"Mann hasn't been honest in decades, so I'm cur...",There have been a dozen re-constructions of Ma...,disagree,1,"Comment: Mann hasn't been honest in decades, s..."
...,...,...,...,...,...
42838,Not trying to spark an argument but a legitima...,Keeping in mind that the Palestinians killed m...,no_disagreement,0,Comment: Not trying to spark an argument but a...
42839,Y'all saw Guilianis hail Mary right? Get his s...,"Same I want these assholes in jail, full stop....",no_disagreement,0,Comment: Y'all saw Guilianis hail Mary right? ...
42840,Why don't I see ads holding Republicans accoun...,"Yeah, I agree with the goal of this post but n...",no_disagreement,0,Comment: Why don't I see ads holding Republica...
42841,How about ... no? This is strange. Community o...,"I know, it feels strange too. We wouldn't hold...",no_disagreement,0,Comment: How about ... no? This is strange. Co...


### Train/Test Split

Make train/val/test split by time order!

In [12]:
# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Determine splitting indexes (ordered by time)
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

### Convert from Pandas DataFrame to Hugging Face Dataset
* Also let's shuffle the training set.
* We put the components train,val,test into a DatasetDict so we can access them later with HF trainer.
* Later we will add a tokenized dataset

In [13]:
X_train_dataset = Dataset.from_pandas(X_train.drop('label', axis = 1))
X_eval_dataset = Dataset.from_pandas(X_eval.drop('label', axis = 1))
X_test_dataset = Dataset.from_pandas(X_test.drop('label', axis = 1))

X_test_dataset

Dataset({
    features: ['comment', 'reply', 'target', 'prompt'],
    num_rows: 4285
})

Shuffle training data --> apparently this helps with performance

In [14]:
X_train_dataset_shuffle = X_train_dataset.shuffle(seed = 42)

In [15]:
dataset = DatasetDict({
    'train' : X_train_dataset_shuffle,
    'val' : X_eval_dataset,
    'test' : X_test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['comment', 'reply', 'target', 'prompt'],
        num_rows: 34274
    })
    val: Dataset({
        features: ['comment', 'reply', 'target', 'prompt'],
        num_rows: 4284
    })
    test: Dataset({
        features: ['comment', 'reply', 'target', 'prompt'],
        num_rows: 4285
    })
})

Check distributions

In [16]:
X_train.target.value_counts(normalize = True)

target
0    0.599463
1    0.400537
Name: proportion, dtype: float64

### Class Weights

* Since our classes are not balanced let's calculate class weights based on inverse value counts
* Convert to pytorch tensor since we will need it

In [17]:
# invert the weights
class_weights = (1/X_train.target.value_counts(normalize = True).sort_index()).to_list()

# make a tensor
class_weights = torch.tensor(class_weights)

# make them sum to one
class_weights = class_weights/class_weights.sum()
class_weights

tensor([0.4005, 0.5995])


## **Load the Model**

Apparently, meta recommends the base version of the model for finetuning [source](https://www.youtube.com/watch?v=YJNbgusTSF0)

* load model with 4bit quantization (as specified in bits and bytes) and prepare model for peft training

In [18]:
model_name = "meta-llama/Llama-3.1-8B" 

In [19]:
torch.cuda.empty_cache()

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    #quantization_config = quantization_config,
    num_labels = 2,
    torch_dtype=torch.bfloat16,  # or float16/float32 depending on your hardware
    device_map="cuda"
)

model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.1-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05

### Freez all parameters, except final head

In [23]:
for name, param in model.named_parameters():
    if "score" not in name:  # "score" is used in HF classification heads
        param.requires_grad = False

In [25]:
# Check trainable parameters
trainable = [n for n, p in model.named_parameters() if p.requires_grad]
print(f"Trainable params: {trainable}")  # Should show only classifier head


Trainable params: ['score.weight']


## **Tokenizer**

### Since LLAMA3 pre-training doesn't have EOS token
* Set the pad_token_id to eos_token_id
* Set pad token ot eos_token

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space = True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

### Update Model Configurations

In [27]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_pt = 1

### Create LLAMA tokenized dataset which will house our train/val parts during the training process but after applying tokenization

In [29]:
MAX_LEN = 512
col_to_delete = ['comment', 'reply', 'prompt']

def llama_preprocessing_function(examples):
    return tokenizer(examples['prompt'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_datasets = tokenized_datasets.rename_column("target", "label")
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/34274 [00:00<?, ? examples/s]

Map:   0%|          | 0/4284 [00:00<?, ? examples/s]

Map:   0%|          | 0/4285 [00:00<?, ? examples/s]

## **Trainer**



### Training Arguments

In [30]:
training_args = TrainingArguments(
    output_dir = 'Llama_3.1_8B_training_2labels_head_only',
    learning_rate = 1e-4,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    num_train_epochs = 2,
    weight_decay = 0.01,
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

In [31]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['val'],
)

In [32]:
train_result = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,1.032900,1.103399
2,1.096800,1.121817


## **Save the model**

In [33]:
metrics = train_result.metrics
max_train_samples = len(X_train)
metrics["train_samples"] = min(max_train_samples, len(X_train))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =         2.0
  total_flos               = 238434159GF
  train_loss               =       1.149
  train_runtime            =  0:52:29.20
  train_samples            =       34274
  train_samples_per_second =      21.767
  train_steps_per_second   =      21.767


In [48]:
trainer.save_model("Llama_3.1_8B_saved_model_2labels_head_only")
tokenizer.save_pretrained("Llama_3.1_8B_saved_model_2labels_head_only") 

('Llama_3.1_8B_saved_model_2labels_head_only/tokenizer_config.json',
 'Llama_3.1_8B_saved_model_2labels_head_only/special_tokens_map.json',
 'Llama_3.1_8B_saved_model_2labels_head_only/tokenizer.json')

***

# **Evaluation**

In [20]:
# reimport the model

tokenizer = AutoTokenizer.from_pretrained("Llama_3.1_8B_saved_model_2labels_head_only")

model = AutoModelForSequenceClassification.from_pretrained(
    "Llama_3.1_8B_saved_model_2labels_head_only",
    torch_dtype=torch.bfloat16,   
    device_map="cuda")



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [24]:
def make_predictions(model,df_test):
    
    model.eval()
    # Convert summaries to a list
    sentences = df_test.prompt.tolist()
    
    # Define the batch size
    batch_size = 32  # You can adjust this based on your system's memory capacity
    
    # Initialize an empty list to store the model outputs
    all_outputs = []
    
    # Process the sentences in batches
    for i in tqdm(range(0, len(sentences), batch_size)):
      # Get the batch of sentences
      batch_sentences = sentences[i:i + batch_size]
    
      # Tokenize the batch
      inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
      # Move tensors to the device where the model is (e.g., GPU or CPU)
      inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}
    
      # Perform inference and store the logits
      with torch.no_grad():
          outputs = model(**inputs)
          all_outputs.append(outputs['logits'])
    final_outputs = torch.cat(all_outputs, dim=0)
    df_test['predictions_ft']=final_outputs.argmax(axis=1).cpu().numpy()
    df_test['predictions_ft']=df_test['predictions_ft'].apply(lambda l:label_dict[l])


make_predictions(model, X_test)

100%|██████████| 134/134 [01:33<00:00,  1.44it/s]
/tmp/ipykernel_8342/2842200323.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions_ft']=final_outputs.argmax(axis=1).cpu().numpy()
/tmp/ipykernel_8342/2842200323.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions_ft']=df_test['predictions_ft'].apply(lambda l:label_dict[l])


In [40]:
X_test.to_csv("output/Llama_3.1_8B_ft_no_LoRa_X_test_2labels.csv", index = False)

In [ ]:
X_test = pd.read_csv("output/Llama_3.1_8B_ft_no_LoRa_X_test_2labels.csv")

## **In Domain Performance**

In [25]:
def get_performance_metrics(df_test, pred_col):
  y_test = df_test.label
  y_pred = df_test[pred_col]

  print("Confusion Matrix:")
  print(confusion_matrix(y_test, y_pred))

  print("\nClassification Report:")
  print(classification_report(y_test, y_pred))

  print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
  print("Accuracy Score:", accuracy_score(y_test, y_pred))

In [43]:
get_performance_metrics(X_test, 'predictions_ft')

Confusion Matrix:
[[ 780  844]
 [ 365 2296]]

Classification Report:
                 precision    recall  f1-score   support

       disagree       0.68      0.48      0.56      1624
no_disagreement       0.73      0.86      0.79      2661

       accuracy                           0.72      4285
      macro avg       0.71      0.67      0.68      4285
   weighted avg       0.71      0.72      0.71      4285

Balanced Accuracy Score: 0.6715645438675413
Accuracy Score: 0.717852975495916


### **To Latex**

In [46]:
y_test = X_test.label
y_pred = X_test['predictions_ft']

report_dict = classification_report(y_test, y_pred, output_dict=True)
df = pd.DataFrame(report_dict).transpose()

# Export to LaTeX table
latex_code = df.to_latex(float_format="%.2f", index=True, caption="Classification Report per Label")
print(latex_code)

\begin{table}
\caption{Classification Report per Label}
\begin{tabular}{lrrrr}
\toprule
 & precision & recall & f1-score & support \\
\midrule
disagree & 0.68 & 0.48 & 0.56 & 1624.00 \\
no_disagreement & 0.73 & 0.86 & 0.79 & 2661.00 \\
accuracy & 0.72 & 0.72 & 0.72 & 0.72 \\
macro avg & 0.71 & 0.67 & 0.68 & 4285.00 \\
weighted avg & 0.71 & 0.72 & 0.71 & 4285.00 \\
\bottomrule
\end{tabular}
\end{table}

